In [ ]:
!pip install transformers datasets tensorflow huggingface_hub -q > /dev/null 2>&1

# roberta-large-bne-finetuned-mldoc

El presente colab se utiliza únicamente para inferencias del modelo ya entrenado.

# Cargar Modelo

In [1]:
import tensorflow as tf
from huggingface_hub import from_pretrained_keras
from transformers import AutoTokenizer, TFRobertaModel
from huggingface_hub import snapshot_download

repo_id = "aatituanav/roberta-base-bne-mldoc-4cat"

# descarga de el repositorio completo
local_dir = snapshot_download(repo_id=repo_id)

# carga el modelo
model = tf.keras.models.load_model(
    local_dir,
    custom_objects={"TFRobertaModel": TFRobertaModel}
)

# carga el tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_dir)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


# Utilización del Modelo
Se clasifican resoluciones de 4 categorias:
*   Infraestructura y Recursos
*   Nombramientos de Personal
*   Aprobaciones de Planes
*   Modificaciones Reglamentarias

#se utilizan las resoluciones del documento proporcionado para las inferencias

In [2]:
def predecir_texto(texto, tokenizer, model, max_length=256):
    # Tokenizar el texto de entrada
    inputs = tokenizer(
        texto,
        truncation=True,
        max_length=max_length,
        padding=True,
        return_tensors="tf"
    )

    # Hacer la predicción
    logits = model.predict({
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"]
    })

    # Clase predicha
    clase_predicha = tf.argmax(logits, axis=1).numpy()[0]

    # Probabilidades de cada clase
    probabilidades = tf.nn.softmax(logits, axis=1).numpy()[0]

    return clase_predicha, probabilidades

In [6]:
texto_ejemplo = """Artículo 1.- Aprobación del Cronograma para Elecciones de Representantes
Estudiantiles del Instituto Superior Tecnológico Yaruquí.  """

# Predecir
clase, probs = predecir_texto(texto_ejemplo, tokenizer, model)

clases = {
    "0": "Infraestructura y Recursos",
    "1": "Nombramientos de Personal",
    "2": "Aprobaciones de Planes",
    "3": "Modificaciones Reglamentarias"
}

print("Resultado de la predicción:")
print(f"Texto: {texto_ejemplo}")
print(f"Clase predicha: {clase} - {clases[str(clase)]}")
print(f"Probabilidades:")
for i, prob in enumerate(probs):
    print(f"   Clase {i} ({clases[str(i)]}): {prob:.4f}")

1/1 [==============================] - 0s 41ms/step
Resultado de la predicción:
Texto: Artículo 1.- Aprobación del Cronograma para Elecciones de Representantes
Estudiantiles del Instituto Superior Tecnológico Yaruquí.  
Clase predicha: 2 - Aprobaciones de Planes
Probabilidades:
   Clase 0 (Infraestructura y Recursos): 0.1822
   Clase 1 (Nombramientos de Personal): 0.1888
   Clase 2 (Aprobaciones de Planes): 0.4442
   Clase 3 (Modificaciones Reglamentarias): 0.1848
